In [ ]:
!pip install pandas scikit-learn matplotlib pydotplus dtreeviz

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("data_tratada.csv")

print("\nDimensões:", df.shape)
print("\nCampos:", df.columns)
print(df.describe())

# Remover registros com alvo ausente
df = df.dropna(subset=['Forma de trabalho ideal'])

# Separar variáveis independentes e alvo
X_dict = df.drop(columns=['Forma de trabalho ideal', 'Cargo atual']).to_dict(orient='records')
vect = DictVectorizer(sparse=False)
X = vect.fit_transform(X_dict)

le = LabelEncoder()
y = le.fit_transform(df['Forma de trabalho ideal'])

# Dividir os dados corretamente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Shape dos dados de treino:", X_train.shape)
print("Shape dos dados de teste:", X_test.shape)

In [ ]:
# Definir modelo com limitação de profundidade e folhas mínimas
treeForma = DecisionTreeClassifier(random_state=0, criterion='entropy', max_depth=5, min_samples_leaf=4)
treeForma.fit(X_train, y_train)

# Avaliação no conjunto de teste
y_pred = treeForma.predict(X_test)
print("Acurácia no teste:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(cnf_matrix, index=[f"Real={c}" for c in le.classes_], columns=[f"Prev={c}" for c in le.classes_])
print(cnf_table)

In [ ]:
# Exibir importâncias dos atributos
importances = pd.Series(treeForma.feature_importances_, index=vect.feature_names_)
importances = importances[importances > 0].sort_values(ascending=False)
print("\nImportância dos atributos:")
print(importances)

# Visualizar graficamente
importances.plot(kind='barh', figsize=(10, 6), title='Importância dos Atributos')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Visualizar a árvore com profundidade limitada
plt.figure(figsize=(20, 10))
tree.plot_tree(treeForma,
               feature_names=vect.feature_names_,
               class_names=le.classes_,
               filled=True,
               rounded=True)
plt.show()